In [ ]:
!wget https://www.dropbox.com/s/9019cpkapt5zm4w/Pic_OralCancer.rar?dl=0

--2023-04-16 10:03:57--  https://www.dropbox.com/s/9019cpkapt5zm4w/Pic_OralCancer.rar?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9019cpkapt5zm4w/Pic_OralCancer.rar [following]
--2023-04-16 10:03:57--  https://www.dropbox.com/s/raw/9019cpkapt5zm4w/Pic_OralCancer.rar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd08045fcb0dd30f7069be11096.dl.dropboxusercontent.com/cd/0/inline/B6S2dicli9yzW9A3jNEX_Vv-cHTcq1_ODbF7b37ucSxtQ9TDg1BjP28yRkddBBEA0IDa05E-PiT-enGEibJL6eYUotiDzhrFfKflA30XlAkypM2h-f7VVKT5XPYGwnnATcV2OkuM0dTMU1Ry3tB5kcr5T2cHEUV4pg6RFIS-FnvXNA/file# [following]
--2023-04-16 10:03:57--  https://ucd08045fcb0dd30f7069be11096.dl.dropboxusercontent.com/cd/0/inline/B6S2dicli9yzW9A3jNEX_Vv-cHTcq1_ODbF7b37ucSxtQ9TDg1BjP28yR

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import cv2
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import vgg16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [ ]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/Pic_OralCancer.rar?dl=0")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
patool: Extracting /content/Pic_OralCancer.rar?dl=0 ...
patool: running /usr/bin/unrar x -- /content/Pic_OralCancer.rar?dl=0
patool:     with cwd='./Unpack_hxdbq6og'
patool: ... /content/Pic_OralCancer.rar?dl=0 extracted to `Pic_OralCancer'.


'Pic_OralCancer'

In [ ]:
# Count number of images in respective classes
#ROOT_DIR = "/content/train"
ROOT_DIR = "/content/Pic_OralCancer"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

image_count=number_of_images.items()
print(image_count)

dict_items([('cancer', 87), ('non-cancer', 44)])


In [ ]:
import os
import random
import cv2
import imgaug.augmenters as iaa

def augment_images(input_path, output_path, cancer_type):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Define augmentations
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            iaa.Fliplr(0.5),
            iaa.Flipud(0.5),
            sometimes(iaa.Affine(rotate=(-45, 45))),
            sometimes(iaa.GaussianBlur(sigma=(0, 3.0))),
            sometimes(iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)),
            sometimes(iaa.Multiply((0.5, 1.5), per_channel=0.5)),
            sometimes(iaa.ContrastNormalization((0.75, 1.5), per_channel=0.5)),
            sometimes(iaa.Grayscale(alpha=(0.0, 1.0))),
        ],
        random_order=True
    )

    for filename in os.listdir(input_path):
        if not filename.endswith('.png') and not filename.endswith('.jpg') and not filename.endswith('.jpeg'):
            continue

        # Load image
        image = cv2.imread(os.path.join(input_path, filename))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Augment images based on cancer type
        if cancer_type == 'cancer':
            n_augmentations = random.randint(4, 5)
        else:
            n_augmentations = random.randint(5, 8)

        images_aug = seq.augment_images([image] * n_augmentations)

        # Save augmented images to output directory
        for i in range(n_augmentations):
            output_filename = os.path.splitext(filename)[0] + f'_aug{i+1}.png'
            cv2.imwrite(os.path.join(output_path, output_filename), images_aug[i], params=[cv2.IMWRITE_PNG_COMPRESSION, 9])


In [ ]:
#Test cancer Images

input_path = '/content/Pic_OralCancer/non-cancer'
output_path = '/content/Pic_OralCancer/non-cancer'
cancer_type = 'non-cancer'

augment_images(input_path, output_path, cancer_type)

In [ ]:
#Test cancer Images
#augment_images("/content/test/cancer", "/content/test/cancer", "cancer")
input_path = '/content/Pic_OralCancer/cancer'
output_path = '/content/Pic_OralCancer/cancer'
cancer_type = 'cancer'

augment_images(input_path, output_path, cancer_type)

In [ ]:
# Count number of images in respective classes
ROOT_DIR = "/content/Pic_OralCancer"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR, dir)))

image_count=number_of_images.items()
print(image_count)

dict_items([('cancer', 476), ('non-cancer', 318)])


In [ ]:
!pip install genetic_algorithm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for genetic_algorithm: filename=genetic_algorithm-1.0.0-py3-none-any.whl size=7637 sha256=75ad473b61f655da7c8609ba36e64ed346a8a4e0ecbd95f3d28f5620a93e17cd
  Stored in directory: /root/.cache/pip/wheels/52/d3/63/fc1fcb6e86b4fdc03ade6b91cc0d767d9221dbb3bb4011b2c8
Successfully built genetic_algorithm


In [ ]:
def datafolder(p,split):
  if not os.path.exists("./"+p):
    os.mkdir("./"+p)

  for dir in os.listdir(ROOT_DIR):
    os.makedirs("./"+p+"/"+dir)
    for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)) ,
                                size =(math.floor(split*number_of_images[dir])-5),
                                replace=False ):
      O = os.path.join(ROOT_DIR,dir,img)  #path
      D = os.path.join("./"+p,dir)
      shutil.copy(O,D)
      os.remove(O)

  else :
    print(f"{p}Folder exists")

In [ ]:
datafolder("train",0.7)
datafolder("val",0.3)

trainFolder exists
valFolder exists


In [ ]:
!pip install lsoptim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement lsoptim (from versions: none)
ERROR: No matching distribution found for lsoptim


In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import random
import tensorflow as tf
from deap import base, creator, tools

# Set the training directory path
train_dir = '/content/train'

# Load and preprocess the data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32)

# Define the evaluation function
def evaluate(individual):
    # Unpack the hyperparameters
    learning_rate, batch_size = individual

    # Define the model
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        input_shape=(224, 224, 3),
        classes=2)

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(train_data.batch(batch_size),
                        epochs=10,
                        verbose=0)

    # Return the validation accuracy as fitness
    return history.history['val_accuracy'][-1],

# Set up the DEAP optimization framework
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("learning_rate", random.uniform, 0.0001, 0.1)
toolbox.register("batch_size", random.randint, 8, 64)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.learning_rate, toolbox.batch_size), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

# Set up the optimization parameters
pop_size = 10
num_generations = 10

# Initialize the population
pop = toolbox.population(n=pop_size)


Found 545 files belonging to 2 classes.
